In [1]:
import numpy as np

from scipy.sparse import load_npz

In [2]:
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics

In [3]:
from imblearn.over_sampling import RandomOverSampler

### Reading data

In [4]:
Xtest = load_npz('features_inga_duplicated.npz')
Xtest

<1719x1676 sparse matrix of type '<class 'numpy.float64'>'
	with 27033 stored elements in Compressed Sparse Row format>

In [5]:
Ytest = np.load('classes_inga_duplicated.npy')
Ytest.shape

(1719, 71)

In [6]:
def predict(clf, params, X, Y, n_splits=5):
    accuracytrain = []
    accuracytest = []
    hammingtrain = []
    hammingtest = []
    f1train = []
    f1test = []
    precisiontrain = []
    precisiontest = []
    recalltrain = []
    recalltest = []

    kf = KFold(n_splits=n_splits)
    for train_idx, test_idx in kf.split(X):
        Xtrain = X[train_idx]
        Xtest = X[test_idx]
        Ytrain = Y[train_idx]
        Ytest = Y[test_idx]

        Ytrainpred_arr = []
        Ytestpred_arr = []

        for i in range(71):
            
            ytrain = Ytrain[:,i]
            ytest = Ytest[:,i]
            
            if np.unique(ytrain).size == 1:
                ytrainpred = np.full(ytrain.shape, ytrain[0])
                ytestpred = np.full(ytest.shape, ytrain[0])
            else:
                ros = RandomOverSampler()
                Xtrain_resampled, ytrain_resampled = ros.fit_sample(Xtrain, ytrain)
                
                gscv = GridSearchCV(clf, params, scoring='accuracy', cv=5, n_jobs=-1)
                gscv.fit(Xtrain_resampled, ytrain_resampled)
                print(gscv.best_score_)
                print(gscv.best_params_)
            
            ytrainpred = gscv.predict(Xtrain)
            ytestpred = gscv.predict(Xtest)
            Ytrainpred_arr.append(ytrainpred)
            Ytestpred_arr.append(ytestpred)

        Ytrainpred = np.array(Ytrainpred_arr).T
        Ytestpred = np.array(Ytestpred_arr).T

        Ytrain = (Ytrain==1).astype('int')
        Ytest = (Ytest==1).astype('int')
        Ytrainpred = (Ytrainpred==1).astype('int')
        Ytestpred = (Ytestpred==1).astype('int')

        accuracytrain.append(metrics.accuracy_score(Ytrain, Ytrainpred))
        accuracytest.append(metrics.accuracy_score(Ytest, Ytestpred))
        hammingtrain.append(1 - metrics.hamming_loss(Ytrain, Ytrainpred))
        hammingtest.append(1 - metrics.hamming_loss(Ytest, Ytestpred))
        f1train.append(metrics.f1_score(Ytrain, Ytrainpred, average='micro'))
        f1test.append(metrics.f1_score(Ytest, Ytestpred, average='micro'))
        precisiontrain.append(metrics.precision_score(Ytrain, Ytrainpred, average='micro'))
        precisiontest.append(metrics.precision_score(Ytest, Ytestpred, average='micro'))
        recalltrain.append(metrics.recall_score(Ytrain, Ytrainpred, average='micro'))
        recalltest.append(metrics.recall_score(Ytest, Ytestpred, average='micro'))

    print('Accuracy: \t \t {} \t {}'.format(np.array(accuracytrain).mean(), np.array(accuracytest).mean()))
    print('Hamming: \t \t {} \t {}'.format(np.array(hammingtrain).mean(), np.array(hammingtest).mean()))
    print('Precision: \t \t {} \t {}'.format(np.array(precisiontrain).mean(), np.array(precisiontest).mean()))
    print('Recall: \t \t {} \t {}'.format(np.array(recalltrain).mean(), np.array(recalltest).mean()))
    print('F1: \t \t \t {} \t {}'.format(np.array(f1train).mean(), np.array(f1test).mean()))

In [7]:
params = {'C': [1.0, 10.0, 100.0]}

In [8]:
predict(LogisticRegression(penalty='l2', solver='liblinear', n_jobs=-1), params, Xtest, Ytest)

/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".forma

0.999038461538
{'C': 1.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".forma

0.950931677019
{'C': 100.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".forma

...

/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".forma

0.971931137725
{'C': 10.0}


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".forma

0.992792109256
{'C': 1.0}
Accuracy: 	 	 0.5202151162790698 	 0.3594989490812936
Hamming: 	 	 0.9819763749516124 	 0.9716682630190254
Precision: 	 	 0.7897918771281505 	 0.7200988532674526
Recall: 	 	 0.9863905918785635 	 0.9277877050720049
F1: 	 	 	 0.8771489692708752 	 0.8104421308732672


/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [9]:
predict(LinearSVC(penalty='l2'), params, Xtest, Ytest)

0.999038461538
{'C': 1.0}
0.94099378882
{'C': 1.0}
0.989819890368
{'C': 1.0}
0.924122807018
{'C': 1.0}
0.995852187029
{'C': 1.0}
0.973404255319
{'C': 1.0}
0.976796407186
{'C': 1.0}
1.0
{'C': 1.0}
0.998867069486
{'C': 1.0}
0.804224207961
{'C': 1.0}
0.962315462315
{'C': 1.0}
0.993140243902
{'C': 1.0}
0.972171532847
{'C': 1.0}
0.995029821074
{'C': 1.0}
0.995791889824
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
0.996174445295
{'C': 1.0}
1.0
{'C': 1.0}
0.994644223412
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
0.998351195383
{'C': 1.0}
0.998082822086
{'C': 1.0}
0.999220576773
{'C': 1.0}
0.998104624716
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
0.975116640747
{'C': 1.0}
0.906439854192
{'C': 1.0}
0.981089258699
{'C': 1.0}
0.998489425982
{'C': 1.0}
0.996598639456
{'C': 1.0}
0.999233128834
{'C': 1.0}
1.0
{'C': 1.0}
1.0
{'C': 1.0}
0.993869731801
{'C': 1.0}
0.998498498498
{'C': 1.0}
0.999618611747
{'C': 1.0}
0.998135719612
{'C': 1.0}
0.992469879518
{'C': 1.0}
